# Imports

Have `py-gordon_1_0_0` isntalled and activated to run this notebook.

In [1]:
import pandas as pd
import numpy as np
import datetime as dt

# Method definition

In [7]:
def createAODDataCassandraHeaders(file_url, simulation_number):
    
    # error code 1 zone
    print("Zone 1...", end=" ")
    simSamId = "EnsNo_1_SimNo_" + str(simulation_number)
    simSam = c3.SimulationSample.get(simSamId)
    sample = c3.NetCDFUtil.openFile(file_url)
    print("done.")
    
    
    # error code 2 zone
    print("Zone 2...", end=" ")
    df_st = pd.DataFrame()
    lat = sample["latitude"][:]
    lon = [x*(x < 180) + (x - 360)*(x >= 180) for x in sample["longitude"][:]]
    tim = sample["time"][:]
    zero_time = dt.datetime(1970,1,1,0,0)
    times = []
    for t in tim:
        target_time = zero_time + dt.timedelta(hours=t)
        times.append(target_time)
            
    df_st["time"] = [t for t in times for n in range(0, len(lat)*len(lon))]
    df_st["latitude"] = [l for l in lat for n in range(0, len(lon))]*len(times)
    df_st["longitude"] = [l for l in lon]*len(times)*len(lat)
    df_st["id"] = round(df_st["latitude"],3).astype(str) + "_" + round(df_st["longitude"],3).astype(str) + "_" + df_st["time"].astype(str).apply(lambda x: x.replace(" ", 'T'))
    print("done.")
    
    
    # error code 3 zone
    print("Zone 3...", end=" ")
    output_records = df_st.to_dict(orient="records")
    c3.GeoSurfaceTimePoint.upsertBatch(objs=output_records)
    print("done.")
    
    print(f"{file_url} had headers created.")
    
    return

# Runs

In [3]:
file_url = "azure://aod-3hourly/jul/bs714a.pb20170711.pp_076.nc"
simulation_number = 76
createAODDataCassandraHeaders(file_url, simulation_number)

Zone 1... done.
Zone 2... 

TypeError: iteration over a 0-d array

In [5]:
file_url = "azure://aod-3hourly/jul/bs714a.pb20170705.pp_076.nc"
simulation_number = 76
createAODDataCassandraHeaders(file_url, simulation_number)

Zone 1... done.
Zone 2... 

TypeError: iteration over a 0-d array

# Records

- `azure://aod-3hourly/jul/bs714a.pb20170711.pp_076.nc` has zero time coordinates
- `azure://aod-3hourly/jul/bs714a.pb20170705.pp_076.nc` has zero time coordinates